In [1]:
from dataset import get_datasets


train_dataset, test_dataset = get_datasets()

In [25]:
import pandas as pd
import torch.nn as nn
from torchvision import models

from dataset import PosterDataset


df = pd.read_csv(PosterDataset.csv_file)

resnet = models.resnet50(pretrained=True)
last_layer_classifier = nn.Sequential(
    nn.Linear(resnet.fc.in_features, len(df.columns) - 2), nn.Sigmoid()
)

/Users/borisleung/anaconda3/envs/stat4012proj/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/borisleung/anaconda3/envs/stat4012proj/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [26]:
import lightning as L
from torch.utils.data import DataLoader


from model import pretrainedModel


model = pretrainedModel(resnet, last_layer_classifier)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)
trainer = L.Trainer(max_epochs=10, fast_dev_run=True)
trainer.fit(
    model,
    train_dataloaders=validation_dataloader,
    val_dataloaders=validation_dataloader,
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.

  | Name             | Type              | Params
-------------------------------------------------------
0 | features         | Sequential        | 23.5 M
1 | class_classifier | Sequential        | 38.9 K
2 | loss_fn          | BCEWithLogitsLoss | 0     
-------------------------------------------------------
23.5 M    Trainable params
0         Non-trainable params
23.5 M    Total params
94.188    Total estimated model params size (MB)
/Users/borisleung/anaconda3/envs/stat4012proj/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_worke

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=1` reached.
